In [ ]:
import polars as pl
import pandas as pd
import preprocessing.feature_engineering as fe
import csv
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, recall_score, precision_score
from sklearn.preprocessing import StandardScaler

In [ ]:
RSEED = 42

In [ ]:
df = pl.read_parquet("./data/2022_data_ST4000DM000.parquet")

filter out faulty data entries:

In [ ]:
fe.search_faulty_drives(df)

In [ ]:

faulty_data = pd.read_csv("./data/faulty_drives.csv").to_list()

In [ ]:
df_filter = df.filter( ~ pl.col("serial_number").is_in(faulty_data))

In [ ]:
#check if faulty entries were dropped
print(df.shape)
print(df_filter.shape)

create DataFrames for modeling

In [ ]:
df_filter = df_filter.to_pandas()

In [ ]:
df_basemodel = df_filter.loc[:,["smart_5_raw","smart_187_raw","smart_188_raw","smart_197_raw","smart_198_raw","failing_in14days"]]

In [ ]:
df_basemodel = df_basemodel.dropna()

In [ ]:
df_basemodel.shape

In [ ]:
df_target = df_basemodel.pop("failing_in14days")

In [ ]:
df_target

train-test-split & scaling

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_basemodel, df_target, train_size= 0.7, random_state=42, stratify=df_target)

In [ ]:
st_scaler = StandardScaler()

In [ ]:
X_train = st_scaler.fit_transform(X_train)

In [ ]:
X_test = st_scaler.transform(X_test)

logistic regression as basemodel

In [ ]:
basemodel = LogisticRegression()

In [ ]:
basemodel.fit(X_train, y_train)

In [ ]:
y_pred = basemodel.predict(X_test)

In [ ]:
train_pred = basemodel.predict(X_train)

In [ ]:
accuracy_score(y_test,y_pred)

In [ ]:
recall_score(y_test, y_pred)

In [ ]:
precision_score(y_test, y_pred)

In [ ]:
matrix = confusion_matrix(y_test, y_pred)

In [ ]:
matrix

In [ ]:
train_matrix = confusion_matrix(y_train, train_pred)
train_matrix